In [6]:
import numpy as np
import pandas as pd
import jams
import os
from tqdm import tqdm
import matplotlib.pyplot as plt
from scipy.stats import hmean

from joblib import Parallel, delayed

In [2]:
TRACK_CSV = '/scratch/work/sonyc/marl/private_datasets/FMA/fma_metadata/fma_metadata/raw_tracks.csv'
track_df = pd.read_csv(TRACK_CSV, index_col='track_id')
print(track_df.columns)
track_df.head(15)

Index(['album_id', 'album_title', 'album_url', 'artist_id', 'artist_name',
       'artist_url', 'artist_website', 'license_image_file',
       'license_image_file_large', 'license_parent_id', 'license_title',
       'license_url', 'tags', 'track_bit_rate', 'track_comments',
       'track_composer', 'track_copyright_c', 'track_copyright_p',
       'track_date_created', 'track_date_recorded', 'track_disc_number',
       'track_duration', 'track_explicit', 'track_explicit_notes',
       'track_favorites', 'track_file', 'track_genres', 'track_image_file',
       'track_information', 'track_instrumental', 'track_interest',
       'track_language_code', 'track_listens', 'track_lyricist',
       'track_number', 'track_publisher', 'track_title', 'track_url'],
      dtype='object')


,album_id,album_title,album_url,artist_id,artist_name,artist_url,artist_website,license_image_file,license_image_file_large,license_parent_id,...,track_information,track_instrumental,track_interest,track_language_code,track_listens,track_lyricist,track_number,track_publisher,track_title,track_url
track_id,,,,,,,,,,,,,,,,,,,,,
2,1.0,AWOL - A Way Of Life,http://freemusicarchive.org/music/AWOL/AWOL_-_...,1,AWOL,http://freemusicarchive.org/music/AWOL/,http://www.AzillionRecords.blogspot.com,http://i.creativecommons.org/l/by-nc-sa/3.0/us...,http://fma-files.s3.amazonaws.com/resources/im...,5.0,...,NaN,0,4656,en,1293,NaN,3,NaN,Food,http://freemusicarchive.org/music/AWOL/AWOL_-_...
3,1.0,AWOL - A Way Of Life,http://freemusicarchive.org/music/AWOL/AWOL_-_...,1,AWOL,http://freemusicarchive.org/music/AWOL/,http://www.AzillionRecords.blogspot.com,http://i.creativecommons.org/l/by-nc-sa/3.0/us...,http://fma-files.s3.amazonaws.com/resources/im...,5.0,...,NaN,0,1470,en,514,NaN,4,NaN,Electric Ave,http://freemusicarchive.org/music/AWOL/AWOL_-_...
5,1.0,AWOL - A Way Of Life,http://freemusicarchive.org/music/AWOL/AWOL_-_...,1,AWOL,http://freemusicarchive.org/music/AWOL/,http://www.AzillionRecords.blogspot.com,http://i.creativecommons.org/l/by-nc-sa/3.0/us...,http://fma-files.s3.amazonaws.com/resources/im...,5.0,...,NaN,0,1933,en,1151,NaN,6,NaN,This World,http://freemusicarchive.org/music/AWOL/AWOL_-_...
10,6.0,Constant Hitmaker,http://freemusicarchive.org/music/Kurt_Vile/Co...,6,Kurt Vile,http://freemusicarchive.org/music/Kurt_Vile/,http://kurtvile.com,http://i.creativecommons.org/l/by-nc-nd/3.0/88...,http://fma-files.s3.amazonaws.com/resources/im...,NaN,...,NaN,0,54881,en,50135,NaN,1,NaN,Freeway,http://freemusicarchive.org/music/Kurt_Vile/Co...
20,4.0,Niris,http://freemusicarchive.org/music/Chris_and_Ni...,4,Nicky Cook,http://freemusicarchive.org/music/Chris_and_Ni...,NaN,http://i.creativecommons.org/l/by-nc-nd/3.0/88...,http://fma-files.s3.amazonaws.com/resources/im...,NaN,...,NaN,0,978,en,361,NaN,3,NaN,Spiritual Level,http://freemusicarchive.org/music/Chris_and_Ni...
26,4.0,Niris,http://freemusicarchive.org/music/Chris_and_Ni...,4,Nicky Cook,http://freemusicarchive.org/music/Chris_and_Ni...,NaN,http://i.creativecommons.org/l/by-nc-nd/3.0/88...,http://fma-files.s3.amazonaws.com/resources/im...,NaN,...,NaN,0,1060,en,193,NaN,4,NaN,Where is your Love?,http://freemusicarchive.org/music/Chris_and_Ni...
30,4.0,Niris,http://freemusicarchive.org/music/Chris_and_Ni...,4,Nicky Cook,http://freemusicarchive.org/music/Chris_and_Ni...,NaN,http://i.creativecommons.org/l/by-nc-nd/3.0/88...,http://fma-files.s3.amazonaws.com/resources/im...,NaN,...,NaN,0,718,en,612,NaN,5,NaN,Too Happy,http://freemusicarchive.org/music/Chris_and_Ni...
46,4.0,Niris,http://freemusicarchive.org/music/Chris_and_Ni...,4,Nicky Cook,http://freemusicarchive.org/music/Chris_and_Ni...,NaN,http://i.creativecommons.org/l/by-nc-nd/3.0/88...,http://fma-files.s3.amazonaws.com/resources/im...,NaN,...,NaN,0,252,en,171,NaN,8,NaN,Yosemite,http://freemusicarchive.org/music/Chris_and_Ni...
48,4.0,Niris,http://freemusicarchive.org/music/Chris_and_Ni...,4,Nicky Cook,http://freemusicarchive.org/music/Chris_and_Ni...,NaN,http://i.creativecommons.org/l/by-nc-nd/3.0/88...,http://fma-files.s3.amazonaws.com/resources/im...,NaN,...,NaN,0,247,en,173,NaN,9,NaN,Light of Light,http://freemusicarchive.org/music/Chris_and_Ni...


In [3]:
JAMS_DIR = '/scratch/ci411/rap_data/jams_test_000'

metric_tags = ['Voicing Recall', 'Voicing False Alarm', 'Raw Pitch Accuracy',\
               'Raw Chroma Accuracy', 'Overall Accuracy']

cref_map = {'Voicing Recall': 'cref_recall', 'Voicing False Alarm': 'cref_false_alarm', \
            'Raw Pitch Accuracy': 'cref_pitch_acc', 'Raw Chroma Accuracy': 'cref_chroma_acc',\
            'Overall Accuracy': 'cref_overall_acc'}

pref_map = {'Voicing Recall': 'pref_recall', 'Voicing False Alarm': 'pref_false_alarm', \
            'Raw Pitch Accuracy': 'pref_pitch_acc', 'Raw Chroma Accuracy': 'pref_chroma_acc',\
            'Overall Accuracy': 'pref_overall_acc'}


jams_df = pd.DataFrame(columns=['track_id', 'cref_recall', 'cref_false_alarm', 'cref_pitch_acc',\
                                'cref_chroma_acc', 'cref_overall_acc', 'pref_recall', 'pref_false_alarm',\
                                'pref_pitch_acc', 'pref_chroma_acc', 'pref_overall_acc'])

for path, subdirs, files in os.walk(JAMS_DIR):
    for file in tqdm(files):
        
        filepath = os.path.join(JAMS_DIR, file)
        loaded_jams = jams.load(filepath)
        
        row_dict = {}
        row_dict['track_id'] = file.split('.')[0]
        for tag in metric_tags:
            if type(loaded_jams['sandbox']['crepe_ref_metrics'])==dict:
                row_dict[cref_map[tag]] = loaded_jams['sandbox']['crepe_ref_metrics'][tag]
            else:
                row_dict[cref_map[tag]] = float('NaN')
            if type(loaded_jams['sandbox']['pyin_ref_metrics'])==dict:    
                row_dict[pref_map[tag]] = loaded_jams['sandbox']['pyin_ref_metrics'][tag]
            else:
                row_dict[pref_map[tag]] = float('NaN')
        jams_df = jams_df.append(row_dict, ignore_index=True)
jams_df

100%|██████████| 62/62 [00:14<00:00,  4.23it/s]


,track_id,cref_recall,cref_false_alarm,cref_pitch_acc,cref_chroma_acc,cref_overall_acc,pref_recall,pref_false_alarm,pref_pitch_acc,pref_chroma_acc,pref_overall_acc
0,000997,0.664903,0,0.492063,0.492063,0.492063,0.218878,0,0.478796,0.478796,0.163475
1,000005,0.715728,0,0.630181,0.630181,0.630181,0.815554,0,0.853407,0.853407,0.726772
2,000705,0.879217,0,0.715996,0.715996,0.715996,0.725756,0,0.684046,0.684567,0.596976
3,000141,0.905606,0,0.747321,0.747321,0.747321,0.891345,0,0.778457,0.778457,0.739417
4,000708,0.784537,0,0.684480,0.684480,0.684480,0.711504,0,0.754572,0.754572,0.624779
...,...,...,...,...,...,...,...,...,...,...,...
57,000704,0.704969,0,0.557143,0.557143,0.557143,0.808860,0,0.672683,0.673503,0.630025
58,000715,0.982170,0,0.918437,0.918437,0.918437,0.889996,0,0.909526,0.909526,0.829414
59,000814,0.845546,0,0.810345,0.810345,0.810345,0.886760,0,0.892857,0.898084,0.851045
60,000667,0.779151,0,0.620077,0.620077,0.620077,0.575937,0,0.605523,0.605523,0.460881


In [4]:
metric_tags = ['Voicing Recall', 'Voicing False Alarm', 'Raw Pitch Accuracy',\
               'Raw Chroma Accuracy', 'Overall Accuracy']

cref_map = {'Voicing Recall': 'cref_recall', 'Voicing False Alarm': 'cref_false_alarm', \
            'Raw Pitch Accuracy': 'cref_pitch_acc', 'Raw Chroma Accuracy': 'cref_chroma_acc',\
            'Overall Accuracy': 'cref_overall_acc'}

pref_map = {'Voicing Recall': 'pref_recall', 'Voicing False Alarm': 'pref_false_alarm', \
            'Raw Pitch Accuracy': 'pref_pitch_acc', 'Raw Chroma Accuracy': 'pref_chroma_acc',\
            'Overall Accuracy': 'pref_overall_acc'}

def agg_jams(jams_dir):
    #define structur of output df
    jams_df = pd.DataFrame(columns=['track_id', 'cref_recall', 'cref_false_alarm', 'cref_pitch_acc',\
                                'cref_chroma_acc', 'cref_overall_acc', 'pref_recall', 'pref_false_alarm',\
                                'pref_pitch_acc', 'pref_chroma_acc', 'pref_overall_acc'])
    
    #iterate through files in directory
    for path, subdirs, files in os.walk(jams_dir):
        for file in tqdm(files):
            
            #define paths and load jams files
            filepath = os.path.join(jams_dir, file)
            loaded_jams = jams.load(filepath)
            
            #establish dict structure and use filename to grab track id
            row_dict = {}
            row_dict['track_id'] = file.split('.')[0]
            
            #extract tags, return NaN if no voicings in reference
            for tag in metric_tags:
                if type(loaded_jams['sandbox']['crepe_ref_metrics'])==dict:
                    row_dict[cref_map[tag]] = loaded_jams['sandbox']['crepe_ref_metrics'][tag]
                else:
                    row_dict[cref_map[tag]] = float('NaN')
                if type(loaded_jams['sandbox']['pyin_ref_metrics'])==dict:    
                    row_dict[pref_map[tag]] = loaded_jams['sandbox']['pyin_ref_metrics'][tag]
                else:
                    row_dict[pref_map[tag]] = float('NaN')
            jams_df = jams_df.append(row_dict, ignore_index=True)
    
    return jams_df


In [ ]:
def append_to_df(filepath, df):
    loaded_jams = jams.load(filepath)
    file = filepath.split('/')[-1].split('.')[0]
            
    #establish dict structure and use filename to grab track id
    row_dict = {}
    row_dict['track_id'] = file.split('.')[0]

    #extract tags, return NaN if no voicings in reference
    for tag in metric_tags:
        if type(loaded_jams['sandbox']['crepe_ref_metrics'])==dict:
            row_dict[cref_map[tag]] = loaded_jams['sandbox']['crepe_ref_metrics'][tag]
        else:
            row_dict[cref_map[tag]] = float('NaN')
        if type(loaded_jams['sandbox']['pyin_ref_metrics'])==dict:    
            row_dict[pref_map[tag]] = loaded_jams['sandbox']['pyin_ref_metrics'][tag]
        else:
            row_dict[pref_map[tag]] = float('NaN')
    jams_df = jams_df.append(row_dict, ignore_index=True)
    return jams_df

def agg_jams_parallel(jams_dir):
    #define structur of output df
    jams_df = pd.DataFrame(columns=['track_id', 'cref_recall', 'cref_false_alarm', 'cref_pitch_acc',\
                                'cref_chroma_acc', 'cref_overall_acc', 'pref_recall', 'pref_false_alarm',\
                                'pref_pitch_acc', 'pref_chroma_acc', 'pref_overall_acc'])
    
    #iterate through files in directory
    for path, subdirs, files in os.walk(jams_dir):
        for file in tqdm(files):
            
            #define paths and load jams files
            filepath = os.path.join(jams_dir, file)
           
    
    return jams_df

In [5]:
fma_small_df = agg_jams('/scratch/ci411/rap_data/jams_fma_small')

  8%|▊         | 617/7837 [02:28<28:52,  4.17it/s]


KeyboardInterrupt: 

In [ ]:
fma_small_df.describe()

In [ ]:
fma_small_df.sample(25)

In [ ]:
plt.figure(figsize=(16,10))

for column in ['cref_recall',  'cref_pitch_acc', 'cref_chroma_acc', 'cref_overall_acc',\
               'pref_recall',  'pref_pitch_acc', 'pref_chroma_acc', 'pref_overall_acc']:
    fma_small_df[column].hist(bins=50, alpha=.6, label=column)
    print (hmean(fma_small_df[column]))
    
plt.legend()